# Experiment 1b - Process tactile image data
* processed tactile data saved into two arrays:
*        feature_SA and features_RA of shape (n_samples, n_pins, n_pins)
* n_pins is number of pins along side of square array (here 19)
* n_samples is here over time during a tapping contact
* values of features are the SA and FA firing rates from module afferent_response.py
* each dataset is collected over 425 positions across each aperiod grating stimulus 
* dataset processing is iterated as a list over stimuli 1 to 7

To run, first edit dir_data to path where data stored.

In [ ]:
import os, sys, pickle, json
import numpy as np

sys.path.append(os.path.abspath('../common'))
from afferent_response import AfferentResponse as ar

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [ ]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-gratings-jrsi2022/experiment_1b"

n_stimuli = 7
n_positions = 424

stimuli = [f'c_{_}' for _ in range(n_stimuli)]
positions = [f'x_{_}' for _ in range(n_positions)]

with open("../common/tip_params/tipA", 'r') as read_file:
    tip_params = json.load(read_file)

In [ ]:
def rotate_pins(pins, theta):
    R = np.array(((np.cos(theta), -np.sin(theta)), (np.sin(theta), np.cos(theta))))
    pins_rot = [np.matmul(R, pin) for pin in pins]
    return np.array(pins_rot)

def locate_pins(pins, n_pins):
    locs = np.zeros((n_pins, n_pins))
    index_xSort = np.argsort(pins[:,0], axis=0)
    
    index_ySort = np.zeros((n_pins, n_pins))
    n = 0
    for k in range(n_pins):
        indices = np.zeros((n_pins,2))
        for l in range(n_pins):
            indices[l,:] = pins[index_xSort[n],:]
            n+=1
        index_ySort[k,:] = np.argsort(indices[:,1], axis=0)

    for k in range(n_pins):
        for l in range(n_pins):
            locs[k,l] = index_xSort[int(index_ySort[k,l])+n_pins*k]
    return locs

def array_pins(pins, locs):
    n1_pins, n2_pins = np.shape(locs)
    pins_array = np.zeros((2, n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            pins_array[:,k,l] = pins[int(locs[k,l]),:]  
    return pins_array

def features(firing, locs, n_samples):
    n1_pins, n2_pins = np.shape(locs)
    features = np.zeros((n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            features[k,l] = np.mean(firing[:n_samples, int(locs[k,l])])   
    return features

In [ ]:
afferents = ar(False, **tip_params)
pins_init = afferents.init_pins(dir_data + rf"/c_0/x_0.mp4")
print(f"n_pins={len(pins_init)}")

firing_SA, firing_RA, pins = [[] for _ in range(3)]
for i in range(n_stimuli):

    firing_SA_j, firing_RA_j, pins_j = [[] for _ in range(3)]
    for j in range(n_positions):
        print(f'{i,j}/{n_stimuli,n_positions}', end=' ')
        
        firing = afferents.firing(dir_data + rf"/{stimuli[i]}/x_{j}.mp4", pins_init)
    
        firing_SA_j.append(np.stack([firing[0][f'{k}'] for k in range(len(firing[0]))]))
        firing_RA_j.append(np.stack([firing[1][f'{k}'] for k in range(len(firing[1]))]))
        pins_j.append(firing[2])
  
    firing_SA.append(firing_SA_j) 
    firing_RA.append(firing_RA_j) 
    pins.append(pins_j)

In [ ]:
theta = np.radians(-6.2)
n_pins = 19
n_samples = int(np.shape(firing_SA[0][0])[1]/2)

pins_rot = [[rotate_pins(pins[i][j], theta) for j in range(n_positions)] for i in range(n_stimuli)]
locs_rot = [[locate_pins(pins_rot[i][j], n_pins) for j in range(n_positions)] for i in range(n_stimuli)]
pins_array = [[array_pins(pins_rot[i][j], locs_rot[i][j]) for j in range(n_positions)] for i in range(n_stimuli)]

save_obj(pins_array, dir_data + r"/pins")

features_SA = [[features(firing_SA[i][j], locs_rot[i][j], n_samples) for j in range(n_positions)] for i in range(n_stimuli)]
features_RA = [[features(firing_RA[i][j], locs_rot[i][j], n_samples) for j in range(n_positions)] for i in range(n_stimuli)]

save_obj(features_SA, dir_data + r"/features_SA")
save_obj(features_RA, dir_data + r"/features_RA")